In [24]:
from scipy.stats import norm
alpha = 0.1

norm.ppf(0.5*alpha, loc=0, scale=1)
#qnorm(0.5*alpha, mean = 0, sd = 1, lower.tail = FALSE, log.p = FALSE)

-1.6448536269514729

In [25]:
%pylab inline
import logging
import numpy as np
import pandas as pd
import xgboost as xgb
from uplift.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import logging
import pickle
from datetime import timedelta
from os.path import join as pjoin
import random


log_format = '[%(asctime)s] %(name)-25s %(levelname)-8s %(message)s'
logging.basicConfig(
    format=log_format,
    level=logging.INFO,
)
logger = logging.getLogger(__name__)


Populating the interactive namespace from numpy and matplotlib


c:\python38\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['norm']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [26]:
# Чтение данных
df_clients = pd.read_csv('data/clients.csv', index_col='client_id')
df_train = pd.read_csv('data/uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('data/uplift_test.csv', index_col='client_id')

# Извлечение признаков

df_clients['first_issue_time'] = \
    (pd.to_datetime(df_clients['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_clients['first_redeem_time'] = \
    (pd.to_datetime(df_clients['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_clients['issue_redeem_delay'] = df_clients['first_redeem_time'] \
    - df_clients['first_issue_time']
df_clients = df_clients.drop(['first_issue_date', 'first_redeem_date'], axis=1)

df_features = pd.DataFrame({
    'gender_M': (df_clients['gender'] == 'M').astype(int),
    'gender_F': (df_clients['gender'] == 'F').astype(int),
    'gender_U': (df_clients['gender'] == 'U').astype(int),
    'age': df_clients['age'],
    'first_issue_time': df_clients['first_issue_time'],
    'first_redeem_time': df_clients['first_redeem_time'],
    'issue_redeem_delay': df_clients['issue_redeem_delay'],
}).fillna(0)

indices_train = df_train.index
indices_learn, indices_valid = train_test_split(df_train.index, test_size=0.3, random_state=123)

X_learn = df_features.loc[indices_train, :]
target_learn = df_train.loc[indices_train, 'target']
treatment_learn = df_train.loc[indices_train, 'treatment_flg']


In [27]:
logger.info('Loading features...')
with open('features.pkl', 'rb') as f:
    features: pd.DataFrame = pickle.load(f)
logger.info('Features are loaded')

logger.info(f'Features shape: {features.shape}')

logger.info('Preparing data sets...')
features.set_index('client_id', inplace=True)


X_train = features.loc[indices_train, :]
X_train = X_train.astype('float32')
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)

[2020-10-14 10:31:17,050] __main__                  INFO     Loading features...
[2020-10-14 10:31:30,158] __main__                  INFO     Features are loaded
[2020-10-14 10:31:30,159] __main__                  INFO     Features shape: (400162, 334)
[2020-10-14 10:31:30,160] __main__                  INFO     Preparing data sets...


In [28]:
print(X_train.shape)

(200035, 333)


In [29]:
from uplift import tree

clf =tree.ExtraTreeClassifier(criterion='uplift_entropy',max_depth=6,max_features=18)
clf = clf.fit(X_train, treatment_learn, target_learn)
tree.export_graphviz(clf, out_file='tree.dot')  


c:\python38\lib\site-packages\numpy\core\_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [30]:
from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

uplift_model = UpliftTreeClassifier(max_depth=5, min_samples_leaf=200, min_samples_treatment=50,
                                    n_reg=100, evaluationFunction='KL', control_name='control')

uplift_model.fit(X_train,
                 treatment=treatment_learn,
                 y=target_learn)

graph = uplift_tree_plot(uplift_model.fitted_uplift_tree, X_train)
Image(graph.create_png())

c:\python38\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


OSError: Could not load shared object file: llvmlite.dll

In [33]:
 import rpy2.robjects as ro
path='C:/Users/anton/Downloads/tools4uplift/R/BinUplift.R'

def function1(input,output):
    r=ro.r
    r.source(path)
    p=r.rtest(input,output)
    return p


a=function1(12,12)   # calling the function with passing arguments

ModuleNotFoundError: No module named 'rpy2'